Ex.1a Symmentric Key Encryption

In [ ]:
def encrypt_caesar(message, key):
    result = ""
    for char in message:
        if char.isalpha():
            if char.isupper():
                result += chr((ord(char) + key - ord('A')) % 26 + ord('A'))
            else:
                result += chr((ord(char) + key - ord('a')) % 26 + ord('a'))
        else:
            result += char
    return result
def decrypt_caesar(encrypted_message, key):
    return encrypt_caesar(encrypted_message, -key)
if __name__ == "__main__":
    print("Caesar Encryption and Decryption")
    message_to_encrypt =input("Enter the Text to be Encrypted")
    caesar_key =int(input("Enter the Key Size"))
    encrypted_message = encrypt_caesar(message_to_encrypt, caesar_key)
    decrypted_message = decrypt_caesar(encrypted_message, caesar_key)
    print("Original Message:", message_to_encrypt)
    print("Encrypted Message:", encrypted_message)
    print("Decrypted Message:", decrypted_message)


Ex.1b Rail_Fence

In [ ]:
def encrypt_rail_fence(plaintext, rails):
    matrix = [['' for _ in range(len(plaintext))] for _ in range(rails)]
    row, col = 0, 0
    for char in plaintext:
        matrix[row][col] = char
        col += 1
        # Change direction when reaching the top or bottom rail
        if row == 0 or row == rails - 1:
            row += 1
        else:
            row -= 1
    # Read the matrix row by row to get the ciphertext
    ciphertext = ''.join(matrix[i][j] for j in range(len(plaintext)) for i in range(rails))
    return ciphertext
def decrypt_rail_fence(ciphertext, rails):
    matrix = [['' for _ in range(len(ciphertext))] for _ in range(rails)]
    row, col = 0, 0
    for char in ciphertext:
        matrix[row][col] = '*'
        col += 1
        # Change direction when reaching the top or bottom rail
        if row == 0 or row == rails - 1:
            row += 1
        else:
            row -= 1
    # Fill the matrix with the ciphertext
    index = 0
    for i in range(rails):
        for j in range(len(ciphertext)):
            if matrix[i][j] == '*' and index < len(ciphertext):
                matrix[i][j] = ciphertext[index]
                index += 1
    # Read the matrix column by column to get the plaintext
    plaintext = ''.join(matrix[i][j] for j in range(len(ciphertext)) for i in range(rails))
    return plaintext
# Example usage
plaintext = "Hello, "
rails = 3
# Encryption
ciphertext = encrypt_rail_fence(plaintext, rails)
print("Encrypted:", ciphertext)
# Decryption
decrypted_text = decrypt_rail_fence(ciphertext, rails)
print("Decrypted:", decrypted_text)



In [ ]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding

def generate_rsa_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
    )
    public_key = private_key.public_key()
    return private_key, public_key

def encrypt_rsa(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def decrypt_rsa(private_key, ciphertext):
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    ).decode()
    return plaintext

# Example usage
private_key, public_key = generate_rsa_key_pair()
message = "Hello, RSA!"

encrypted_message = encrypt_rsa(public_key, message)
print("Encrypted Message:", encrypted_message.hex())

decrypted_message = decrypt_rsa(private_key, encrypted_message)
print("Decrypted Message:", decrypted_message)


In [ ]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.backends import default_backend

def generate_dh_key_pair():
    parameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

def serialize_key(key):
    return key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

def deserialize_key(serialized_key):
    return serialization.load_pem_public_key(serialized_key, backend=default_backend())

def derive_shared_key(private_key, peer_public_key):
    shared_key = private_key.exchange(peer_public_key)
    return shared_key

# Example usage
alice_private_key, alice_public_key = generate_dh_key_pair()
bob_private_key, bob_public_key = generate_dh_key_pair()

# Serialize public keys
serialized_alice_public_key = serialize_key(alice_public_key)
serialized_bob_public_key = serialize_key(bob_public_key)

# Deserialize public keys
alice_public_key = deserialize_key(serialized_bob_public_key)
bob_public_key = deserialize_key(serialized_alice_public_key)

# Derive shared keys
alice_shared_key = derive_shared_key(alice_private_key, bob_public_key)
bob_shared_key = derive_shared_key(bob_private_key, alice_public_key)

print("Alice's Shared Key:", alice_shared_key.hex())
print("Bob's Shared Key:", bob_shared_key.hex())


In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import ec

def generate_key_pair():
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    public_key = private_key.public_key()
    return private_key, public_key

def sign_message(private_key, message):
    signature = private_key.sign(
        message.encode(),
        ec.ECDSA(hashes.SHA256())
    )
    return signature

def verify_signature(public_key, message, signature):
    try:
        public_key.verify(
            signature,
            message.encode(),
            ec.ECDSA(hashes.SHA256())
        )
        return True
    except Exception:
        return False

# Example usage
private_key, public_key = generate_key_pair()
message = "This is a signed message."

# Signing
signature = sign_message(private_key, message)
print("Signature:", signature.hex())

# Verifying
is_valid = verify_signature(public_key, message, signature)
print("Signature is valid:", is_valid)
